# Select unlabeled documents

In [6]:
import os
import shutil

In [7]:
db_path = "../../Datasets/Tobacco 800 Dataset/tobacco800"
annotated_images = "../../Old_Data/Annotated_Images"
ann_img_list = os.listdir(annotated_images)
images_list = os.listdir(db_path)
print(len(ann_img_list))
print(len(images_list))

## Remove annotated images ##
for img_f in ann_img_list:
    file = img_f+".png"
    images_list.remove(file)

print(len(images_list))


src_path = db_path
dst_path = "../../Data/Images_2_Annotate/"

for file in images_list:
    src_file = os.path.join(src_path, file)
    dst_file = os.path.join(dst_path, file)
    shutil.copy2(src_file, dst_file)



500
1292
792
